In [1]:
import pandas as pd
import random
base = 'trump_tweets.csv'

data = pd.read_csv(base)
data = data[["is_retweet","text"]]

FileNotFoundError: [Errno 2] No such file or directory: 'trump_tweets.csv'

In [ ]:
IMPORTANT = 1
ABSTAIN = 0

In [ ]:
data

,is_retweet,text
0,False,From Donald Trump: Wishing everyone a wonderfu...
1,False,Trump International Tower in Chicago ranked 6t...
2,False,Wishing you and yours a very Happy and Bountif...
3,False,Donald Trump Partners with TV1 on New Reality ...
4,False,"--Work has begun, ahead of schedule, to build ..."
...,...,...
35214,False,".@ICEgov HSI agents and ERO officers, on behal..."
35215,False,Thank you @GOPLeader Kevin McCarthy! Couldn’t ...
35216,False,"As I made very clear today, our country needs ..."
35217,True,"RT @Scavino45: “Utilities cutting rates, cite ..."


In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(data,train_size=0.8)

In [ ]:
from snorkel.labeling import labeling_function,PandasLFApplier,LFAnalysis

In [ ]:
@labeling_function()
def label1(x):
    return IMPORTANT if "vacina" in x.text.lower() else ABSTAIN


@labeling_function()
def label2(x):
    return IMPORTANT if "bolsonaro" in x.text.lower() else ABSTAIN


@labeling_function()
def label3(x):
    return IMPORTANT if "pandemia" in x.text.lower() else ABSTAIN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
lfs = [label1, label2, label3]

In [ ]:
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

/home/vitorararuna/.local/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 17609/17609 [00:01<00:00, 17353.86it/s]


In [ ]:
L_train

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [ ]:
LFAnalysis(L=L_train,lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
label1,0,"[0, 1]",1.0,1.0,0.0
label2,1,"[0, 1]",1.0,1.0,0.0
label3,2,"[0, 1]",1.0,1.0,0.0


In [ ]:
from snorkel.labeling.model import MajorityLabelVoter,LabelModel

In [ ]:
label_model = LabelModel(cardinality=2,verbose=True)
label_model.fit(L_train=L_train,n_epochs=500,log_freq=100,seed=123)

INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|██████████| 500/500 [00:02<00:00, 247.30epoch/s]
INFO:root:Finished Training


In [ ]:
df_train.head()

,is_retweet,text
25788,False,"""@KimberlySyring: @realDonaldTrump @BrandiGlan..."
17814,False,Lightweight @AGSchneiderman is driving busines...
4471,False,"""If other countries benefit from our armed for..."
15827,False,Practice positive thinking--this will keep you...
21169,False,"""@linablue: @realDonaldTrump I love the fact y..."


In [ ]:
df_train['Labels'] = label_model.predict(L=L_train,tie_break_policy="abstain")

/tmp/ipykernel_8666/1145111523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Labels'] = label_model.predict(L=L_train,tie_break_policy="abstain")


In [ ]:
df_train.query('Labels == 0')

,is_retweet,text,Labels
25788,False,"""@KimberlySyring: @realDonaldTrump @BrandiGlan...",0
17814,False,Lightweight @AGSchneiderman is driving busines...,0
4471,False,"""If other countries benefit from our armed for...",0
15827,False,Practice positive thinking--this will keep you...,0
21169,False,"""@linablue: @realDonaldTrump I love the fact y...",0
...,...,...,...
4485,False,We left Iraq and it is quickly falling apart--...,0
8817,False,"""@Johns81366: @realDonaldTrump Mr. Trump, Even...",0
11562,False,She’s back! Champion @Joan_Rivers returns to t...,0
10913,False,3. You should tweet your pick for MVP using t...,0
